In [1]:
%pip install odc-stac -q
#%pip install mapclassify -q
#%pip install folium -q


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import os
from io import StringIO
import geopandas as gpd
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

from pystac_client import Client

from odc.stac import configure_rio, load
import rioxarray

from geodata_fetch.harvest import DataHarvester
from gis_utils.dataframe import get_bbox_from_geodf

In [3]:
configure_rio(cloud_defaults=True, aws={"aws_unsigned": True})

os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'

In [4]:
big_test_area = False

if big_test_area:
    geom_filepath = "/workspace/notebooks/sandbox/data/input-data/onebasin/Surface Water Water Resource Plan Areas/Surface Water Water Resource Plan Areas.shp"
    gdf = gpd.read_file(geom_filepath)
    geom_gdf = gdf.loc[gdf['SWWRPACODE'] == 'SW1']

    geom_crs = geom_gdf.to_crs("EPSG:4326") #needs to be 4326 to work with pystac_client for searching
    geom = geom_crs.geometry
    bbox = list(geom_crs.total_bounds)

    gdf_lon = None
    gdf_lat = None
else:
    geojson = {
    'body': {
        "type": "FeatureCollection",
        "name": "dissolved-boundaries",
        "crs": {
            "type": "name",
            "properties": {
                "name": "urn:ogc:def:crs:OGC:1.3:CRS84" 
            }
        },
        "features": [
            {
                "type": "Feature",
                "properties": {
                    "fid": 1
                },
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [116.26012130269045, -29.225295369642396],
                            [116.261724812149055, -29.241374854584375],
                            [116.283751968396274, -29.256813692452539],
                            [116.284342735038919, -29.268250184258388],
                            [116.292247755352392, -29.265992437426529],
                            [116.292360282331941, -29.293057573630019],
                            [116.314865678242256, -29.293523728033122],
                            [116.326259034921833, -29.293033039128805],
                            [116.326315298411629, -29.305397680579894],
                            [116.355065941687045, -29.307016748931797],
                            [116.355065941687045, -29.306575187382712],
                            [116.383366477044206, -29.307384715430175],
                            [116.384322956370426, -29.290407813444993],
                            [116.387586238777402, -29.282629879611861],
                            [116.386517232471661, -29.259807919053017],
                            [116.359201308185533, -29.259488866292969],
                            [116.359229439930417, -29.259243440415627],
                            [116.35242155766754, -29.259292525638209],
                            [116.352140240218716, -29.220237788279107],
                            [116.302234524787593, -29.223503148505326],
                            [116.281388901825679, -29.2239696200396],
                            [116.26012130269045, -29.225295369642396]
                        ]
                    ]
                }
            }
        ]
    }
}
    req = geojson
    geojson_data = req['body']
    gdf = gpd.read_file(StringIO(json.dumps(geojson_data)))
    geom = gdf.geometry 
    bbox = get_bbox_from_geodf(geojson_data)

    gdf_lon = None
    gdf_lat = None

In [5]:
local=True
map_results = False

notebook_key = "localjupyter"

propertyName = "test"
output_type = "overlay"
colormap = "gist_earth"

dem_layer = "DEM Global"

In [6]:
output_tiff_directory = f"/tmp/{notebook_key}"
if local:
    output_tiff_directory = "/workspace/notebooks/sandbox/data/slga_testing"

# Ensure the storage directory exists
os.makedirs(output_tiff_directory, exist_ok=True)

In [7]:
#papermill_description=processing_input_harvest_params

data_mask=True
add_buffer = False
resample=False

crs = 3857

target_res = 1
target_proj = "EPSG:3857"

date_start = None
date_end = None

time_intervals = 0

target_sources = {
    "DEM Global": dem_layer
}

json_data = {
    "property_name":propertyName,
    "outpath": output_tiff_directory,
    "data_mask": data_mask,
    "target_res": str(target_res),
    "target_crs": target_proj,
    "date_start": date_start if date_start is not None else "2022-10-01",
    "date_end": date_end if date_end is not None else "2022-11-30",
    "target_centroid_lat": gdf_lat,
    "target_centroid_lng": gdf_lon,
    "time_intervals": time_intervals,
    "target_sources": target_sources,
    "target_bbox": bbox,
    "add_buffer": add_buffer,
    "resample": resample
}

# Converting dictionary to JSON formatted string
data = json.dumps(json_data, indent=4)

# Create a file-like object from JSON string
json_file_like = StringIO(data)

In [8]:
print(data)

{
    "property_name": "test",
    "outpath": "/workspace/notebooks/sandbox/data/slga_testing",
    "data_mask": true,
    "target_res": "1",
    "target_crs": "EPSG:3857",
    "date_start": "2022-10-01",
    "date_end": "2022-11-30",
    "target_centroid_lat": null,
    "target_centroid_lng": null,
    "time_intervals": 0,
    "target_sources": {
        "DEM Global": "DEM Global"
    },
    "target_bbox": [
        116.26012130269045,
        -29.307384715430175,
        116.3875862387774,
        -29.220237788279107
    ],
    "add_buffer": false,
    "resample": false
}


In [11]:
geom.explore()

In [9]:
harvester = DataHarvester(json_file_like, geom)
harvester.run()

source type is DEM Global
processing DEM Global:
checkpoint in harvest 1
checkpoint in harvest 2
DEM Global


KeyboardInterrupt: 

In [ ]:
#gdf.explore(column="SWWRPACODE")

In [ ]:
#geom.explore()

In [ ]:
# url = "https://earth-search.aws.element84.com/v1/"
# collection = ["cop-dem-glo-30"]

# start_date = "2024-04-01"
# end_date = "2024-05-01"

# resolution = 90 #this is not the DEM native resolution, only for speeding up testing
# crs = 3857

In [ ]:
# catalog = Client.open(url)

# query = catalog.search(
#     collections=collection,
#     bbox=bbox
# )

In [ ]:
# items = list(query.items())
# print(f"Found {len(items):d} datasets")

# #convert the fetched stac items to a geojson featurecollection
# stac_json = query.item_collection_as_dict()


In [ ]:

# #lazily combine
# xx = load(
#     items,
#     crs=f"epsg:{crs}",
#     resolution = resolution,
#     bbox = bbox,
#     chunksize = (2000,2000)
# ).squeeze()

# # data is not loaded yet
# xx

In [ ]:
# xx = xx.load()

In [ ]:
# data = xx.data
# data.plot()

In [ ]:
# # test saving the xarray dataset as a geotiff

# # convert original polygon to same CRS as raster

# clip_geom = geom_gdf.to_crs(f"EPSG:{crs}")

# mask_raster = data.rio.clip(clip_geom.geometry.values, clip_geom.crs, drop=False)

# mask_raster.plot()

In [ ]:

# test_save = mask_raster.rio.to_raster(f"{output_tiff_directory}/test_save.tif", driver="COG")